## 4.4 Modèle générique

#### Code source de l'ORM simplifié

In [1]:
import sqlite3
import builtins

class ORM():
    def __init__(self, dbname):
        self.__dbname = dbname
        self.__conn = sqlite3.connect(self.__dbname)

        # liste des classes métier gérées par l'ORM
        data = self.execute("SELECT id,name FROM entity").fetchall()
        self.__entities = { e[1]:e[0] for e in data}

    #
    # exécution d'une requête SQL quelconque
    #
    def execute(self, sql, args=()):
        r = self.__conn.cursor().execute(sql,args)
        self.__conn.commit()
        return r

    #
    # création éventuelle d'une classe dans la base
    #
    def create_entity(self, entity):
        
        # on vérifie si la classe est déjà renseignée dans la base
        name = entity.__name__
        sql = "SELECT id FROM entity WHERE name=?"
        id = self.execute(sql,(name,)).fetchone()

        # si la classe n'est pas encore dans la base...
        if not id:
            # ... on la crée
            self.execute("INSERT INTO entity (name) VALUES (?)",(name,))
            id = self.execute(sql,(name,)).fetchone()
            self.__entities[name] = id[0]

        # informations sur ses classes mères
        self.__create_inheritance_chain(entity)
        
        # informations sur ses attributs
        self.__create_attributes(entity)
        return id[0]

    #
    # création des informations sur la chaîne d'héritage d'une classe
    #
    def __create_inheritance_chain(self,entity):
        ancestors = [c.__name__ for c in entity.superclasses()]
        
        # boucle sur les classes mères
        if len(ancestors) > 1:
            for n in range(len(ancestors)-1):

                # on vérifie si l'information existe déjà
                request = "SELECT COUNT(*) FROM inheritance WHERE superclass_id=? AND subclass_id=?"
                args = (self.__entities[ancestors[n+1]],self.__entities[ancestors[n]])
                count = self.execute(request,args).fetchone()[0]

                # si l'information n'existe pas encore...
                if not count:
                    # création de l'information sur un couple classe mère / classe fille
                    req = "INSERT INTO inheritance (superclass_id,subclass_id) VALUES (?,?)"
                    self.execute(req,args)
                    
    #
    # création éventuelle d'une liste attributs dans la base
    #
    def __create_attributes(self, entity):
        attrs = entity.attributes()
        mock = entity()
        for attr_name in attrs:
            self.__create_attribute(entity,attr_name,type(getattr(mock,attr_name)).__name__)

    #
    # création éventuelle d'un attribut dans la base
    #
    def __create_attribute(self, cls, name, _type):
        
        # on vérifie si l'attribut existe déjà
        class_name = cls.__name__
        class_id = self.execute("SELECT id FROM entity WHERE name=?",(class_name,)).fetchone()[0]
        sql = "SELECT id FROM attribute WHERE class_id=? AND name=?"
        id = self.execute(sql,(class_id,name)).fetchone()
        
        # si l'attribut n'existe pas encore...
        if not id:
            # ...on le crée
            request = "INSERT INTO attribute (class_id,name,type) VALUES (?,?,?)"
            self.execute(request, (class_id, name, _type))
            id = self.execute(sql,(class_id,name)).fetchone()
        return id[0]

    #
    # récupération de l'id d'un attribut
    #
    def __get_attribute_id(self, class_name, attribute_name):
        request = "SELECT id FROM attribute WHERE name = ? AND class_id = " + \
                "(SELECT id FROM entity WHERE name = ?)"
        return self.execute(request, (attribute_name, class_name)).fetchone()[0]

    #
    # enregistrement d'un objet dans la base (update ou insert)
    #
    def persist(self, o):
        self.update(o) if hasattr(o,'id') else self.insert(o)

    #
    # mise à jour d'un objet dans la base
    #
    def update(self,o):
        class_name = o.__class__.__name__
        attributes = {k: getattr(o,k) for k in o.__class__.attributes() if not k == 'id'}   
        
        # pour une mise à jour un objet doit posséder un attribut id
        if not hasattr(o,'id'): return # un raise serait plus avisé...

        # boucle sur les attributs de l'objet
        object_id = o.id
        for k in attributes.keys():
            
            # mise à jour de la valeur d'un attribut
            attribute_id = self.__get_attribute_id(class_name,k)
            request = "UPDATE `value` SET value=? WHERE object_id=? AND attribute_id = ?"
            self.execute(request,(attributes[k],object_id, attribute_id))

    #
    # création d'un objet dans la base
    #
    def insert(self,o):
        class_name = o.__class__.__name__
        attributes = {k: getattr(o,k) for k in o.__class__.attributes()}

        # pour une création un objet ne doit pas posséder d'attribut id
        if hasattr(o,'id'): return # un raise serait plus avisé...

        # id de l'objet créé
        object_id = 1 + self.execute("SELECT MAX(object_id) FROM value").fetchone()[0] or 0
        
        # boucle sur les attributs de l'objet
        for k in attributes.keys():

            # création de la valeur d'un attribut
            attribute_id = self.__get_attribute_id(class_name,k)
            request = "INSERT INTO `value` (object_id,attribute_id,value) VALUES (?,?,?)"
            self.execute(request,(object_id, attribute_id, attributes[k]))
        
        # renseignement de l'id de l'objet créé
        o.id = object_id

    #
    # suppression d'un objet dans la base
    #
    def delete(self,o):
        if hasattr(o,'id'):
            self.execute("DELETE FROM `value` WHERE object_id = ?",(o.id,))
            delattr(o,'id')

    #
    # récupération d'un objet ou d'une liste depuis la base
    #
    def query(self, cls, id=None, polymorphic=False):
        
        # pour une requête polymorphique il faut requêter sur l'ensemble des classes filles
        if polymorphic:
            data = self.execute("SELECT superclass_id, subclass_id FROM inheritance").fetchall()
            inheritance = { c[0]:c[1] for c in data}
            superid = self.__entities[cls.__name__]
            subclasses = [superid]
            while superid in inheritance.keys():
                subclasses.append(inheritance[superid])
                superid = inheritance[superid]
        else:
            subclasses = [self.__entities[cls.__name__]]

        # construction de la requête
        sql = "SELECT value.object_id, entity.name, attribute.name, value.value, attribute.type " + \
              "FROM entity, attribute, value " + \
              "WHERE attribute.class_id = entity.id " + \
              "AND value.attribute_id = attribute.id " + \
              "AND entity.id in ({})".format(','.join(['?']*len(subclasses)))
        args = [sc for sc in subclasses]
        if id:
            sql = sql + " AND value.object_id = ?"
            args.append(id)
        
        # exécution de la requête
        r = self.__conn.cursor().execute(sql,args).fetchall()
        
        # construction d'un dictionnaire indexé par l'id des objets retournés
        g = globals()
        objdict = {}
        for (id, classname, attname, value, type) in r:
            if not id in objdict.keys():
                objdict[id] = g[classname]()
                setattr(objdict[id],'id',int(id))
            setattr(objdict[id],attname,getattr(builtins,type)(value))
        
        # on retourne une liste
        return list(objdict.values())

orm = ORM('generic_orm.sqlite')

#### Création des tables

In [2]:
sql = "CREATE TABLE IF NOT EXISTS entity (" + \
        "`id` INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, " + \
        "`name` TEXT NOT NULL UNIQUE" + \
    ")"
orm.execute(sql)

sql = "CREATE TABLE IF NOT EXISTS attribute (" + \
        "`id` INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, " + \
        "`class_id` INTEGER NOT NULL, " + \
        "`name` TEXT NOT NULL, " + \
        "`type` TEXT NOT NULL," + \
        "FOREIGN KEY(`class_id`) REFERENCES `entity.id` " + \
    ")"
orm.execute(sql)

sql = "CREATE TABLE IF NOT EXISTS value (" + \
        "`object_id` INTEGER NOT NULL, " + \
        "`attribute_id` INTEGER NOT NULL, " + \
        "`value` TEXT, " + \
        "FOREIGN KEY(`attribute_id`) REFERENCES `attribute.id`, " + \
        "PRIMARY KEY(`object_id`,`attribute_id`) " + \
    ")"
orm.execute(sql)

sql = "CREATE TABLE IF NOT EXISTS inheritance(" + \
        "superclass_id INTEGER NOT NULL, " + \
        "subclass_id INTEGER NOT NULL, " + \
        "FOREIGN KEY(`superclass_id`) REFERENCES `entity.id`, " + \
        "FOREIGN KEY(`subclass_id`) REFERENCES `entity.id`, " + \
        "PRIMARY KEY(`superclass_id`,`subclass_id`) " + \
    ")"
orm.execute(sql)

pass

#### Classe de base des classes métier

In [3]:
class Base():
    
    #
    # renvoie la liste des attributs d'une classe
    #
    @classmethod
    def attributes(cls):
        attrs = [k for k in cls.__dict__ if not k.startswith('__') and not type(getattr(cls,k)).__name__ == 'method' ]
        if hasattr(cls.__bases__[0],'attributes'):
            attrs = attrs + cls.__bases__[0].attributes()
        return attrs

    #
    # renvoie la liste des classes mères d'une classe
    #
    @classmethod
    def superclasses(cls):
        ancestors = [cls]       
        if hasattr(cls.__bases__[0],'superclasses') and not cls.__bases__[0] == Base:
            ancestors = ancestors + cls.__bases__[0].superclasses()
        return ancestors

    #
    # constructeur des classes métier
    #
    def __init__(self,**kwargs):
        attributes = self.__class__.attributes()
        for k in attributes:
            if k in kwargs.keys():
                setattr(self,k,kwargs[k])

#### Définition des classes métier

In [4]:
#
# N.B. l'ORM simplifié fonctionne uniquement si les attributs sont déclarés au niveau de la classe
# avec une valeur par défaut destinée à indiquer son type. L'héritage multiple n'est pas pris en
# compte, ni la gestion de relations entre classes...
#
class Person(Base):
    first_name = ''
    last_name = ''
    
    def __str__(self):
        return "{}: {} {}".format(self.__class__.__name__, self.first_name,self.last_name)
    
class Developer(Person):
    login = ''
    pwd = ''

    def __str__(self):
        return "{} (login,pwd)=({},{})".format(super().__str__(), self.login,self.pwd)
    
class Player(Person):
    pseudo = ''
    score = 0

    def __str__(self):
        return "{} (pseudo,score)=({},{})".format(super().__str__(), self.pseudo,self.score)
    
class VIP(Player):
    email = ''
    preferences = ''

    def __str__(self):
        return "{} {} prefs={}".format(super().__str__(), self.email,self.preferences)


#### Description des classes dans la base

In [5]:
orm.create_entity(Person)
orm.create_entity(Developer)
orm.create_entity(Player)
orm.create_entity(VIP)
pass

#### Création d'objets métier et enregistrement dans la base

In [6]:
raymond = Person(first_name='Raymond',last_name='Deubaze')
raymond.id = 1
orm.persist(raymond)

In [7]:
p = Player(first_name='Jean', last_name='Bonnot', pseudo='jbt', score=0)
p.id = 2
orm.persist(p)

In [8]:
dev = Developer(first_name='Maud',last_name='Zarella', login="mozarella", pwd="di Bufala")
dev.id = 3
orm.persist(dev)

In [9]:
vip = VIP(first_name="Kelly", last_name="Diotte", pseudo="kde", score=0, email='kelly@gmail.com', preferences='')
vip.id = 4
orm.persist(vip)

In [10]:
elsa = Person(first_name='Elsa',last_name='Plique')
elsa.id = 5
orm.persist(elsa)

In [11]:
jean = Person(first_name='Jean',last_name='Peuplu')
jean.id = 6
orm.persist(jean)

#### Récupération des objets d'une classe

In [12]:
# Requête fetchall()
people = orm.query(Person)
for p in people:
    print(p)

Person: Raymond Deubaze
Person: Elsa Plique
Person: Jean Peuplu


#### Récupération d'un objet à partir de son identifiant

In [13]:
# Requête fetch by id
people = orm.query(Person,1)
for p in people:
    print(p)

Person: Raymond Deubaze


#### Récupération d'une liste polymorphique

In [14]:
# Requête polymorphique
people = orm.query(Person,None,True)
for p in people:
    print(p)

Person: Raymond Deubaze
Player: Jean Bonnot (pseudo,score)=(jbt,0)
VIP: Kelly Diotte (pseudo,score)=(kde,0) kelly@gmail.com prefs=
Person: Elsa Plique
Person: Jean Peuplu
